Train gradient boosting models on the data.

# 1. Load data and packages

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from handyspark import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import GBTClassifier

Load the modeling helper functions

In [5]:
%run Users/tw2567@columbia.edu/Udacity/churn_prediction/util

In [6]:
raw_data = spark.read.parquet('s3://tianyi-wang-data-science-projects/churn-prediction-2020/features1')\
           .withColumnRenamed('last_status_cancelled','label')\
           .drop('userId')
features_col = [i for i in raw_data.columns if i != 'label']
assembler = VectorAssembler(inputCols=features_col,outputCol="features")
data = assembler.transform(raw_data).select('features','label')

In [7]:
feature_names = assembler.getInputCols()

In [8]:
display(data.groupby('label').count())

label,count
1,5005
0,17272


# 2 Check whether it's better to upsample the positive group

### 2.1 Triple the size of positive samples

In [11]:
auc = tune_sampling_parameters(data, GBTClassifier(), 3, 1, num_folds=3)
auc

Minority -- training size went from 3503.5 to 10510.5
Majority -- training size went from 12090.4 to 12090.4
fold 0
10424
12000
fold 1
10422
11967
fold 2
10598
11970
Out[86]: [0.7631292471854523, 0.7708183934321681, 0.7635562607392208]

### 2.2 Double the size of positive samples

In [13]:
auc = tune_sampling_parameters(data, GBTClassifier(), 2, 1, num_folds=3)
auc

Minority -- training size went from 3503.5 to 7007.0
Majority -- training size went from 12090.4 to 12090.4
fold 0
6865
11969
fold 1
6880
11961
fold 2
7048
11951
Out[87]: [0.767649104893028, 0.7599373082524284, 0.7639823909907206]

### 2.3 Postive group size * 1.5

In [15]:
auc = tune_sampling_parameters(data, GBTClassifier(), 1.5, 1, num_folds=3)
auc

Minority -- training size went from 3503.5 to 5255.25
Majority -- training size went from 12090.4 to 12090.4
fold 0
5172
11966
fold 1
5319
11985
fold 2
5084
11945
Out[88]: [0.7628643195979412, 0.7659123433542293, 0.7618465030126286]

We decide to triple the size of the positive group.

### 2.4 Don't upsample

In [18]:
auc = tune_sampling_parameters(data, GBTClassifier(), 1, 1, num_folds=3)
auc

Minority -- training size went from 3503.5 to 3503.5
Majority -- training size went from 12090.4 to 12090.4
fold 0
3497
11968
fold 1
3558
11965
fold 2
3472
11956
Out[89]: [0.7603496842257487, 0.7652316327703625, 0.7599662679482291]

# 3. Tune model parameters

In [20]:
gbt = GBTClassifier()
paramGrid = ParamGridBuilder()\
            .addGrid(gbt.maxDepth, [5, 8])\
            .addGrid(gbt.stepSize, [0.01, 0.05, 0.1])\
            .addGrid(gbt.subsamplingRate, [0.8, 1])\
            .build()
cv_model = tune_model_parameter(data, 3, 1, gbt, paramGrid)

In [21]:
cv_model.avgMetrics

Out[91]: [0.7494699211227707,
 0.7587866758058118,
 0.7820266941321453,
 0.7826277828102545,
 0.7938872000580273,
 0.7939644473491756,
 0.7846681647971381,
 0.7863434389274575,
 0.819330241759582,
 0.8234555540720561,
 0.8391316287455748,
 0.8415929041269218]

In [22]:
cv_model.bestModel.extractParamMap()

Out[98]: {Param(parent='GBTClassifier_ee072705483e', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): False,
 Param(parent='GBTClassifier_ee072705483e', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10,
 Param(parent='GBTClassifier_ee072705483e', name='featureSubsetStrategy', doc='The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].'): 'all',
 Param(parent='GBTClassifier_ee072705483e', name='featuresCol', doc='features column name'): 'features',
 Param(parent='GBTClassifier_ee072705483e', name='labelCol', doc='label column name'): 'label',
 Param(parent='GBTClassifier_ee072705483e', name='lossType', doc='Loss function which GBT tries to minimize (case-insensitive). Supported options: logistic'): 'logistic',
 Param(parent='GBTClassifier_ee072705483e', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be at least 2 and at least number of categories for any categorical feature.'): 32,
 Param(parent='GBTClassifier_ee072705483e', name='maxDepth', doc='Maximum depth of the tree. (Nonnegative) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 8,
 Param(parent='GBTClassifier_ee072705483e', name='maxIter', doc='maximum number of iterations (>= 0)'): 20,
 Param(parent='GBTClassifier_ee072705483e', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation.'): 256,
 Param(parent='GBTClassifier_ee072705483e', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent='GBTClassifier_ee072705483e', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Must be at least 1.'): 1,
 Param(parent='GBTClassifier_ee072705483e', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='GBTClassifier_ee072705483e', name='seed', doc='random seed'): 3504127614838123891,
 Param(parent='GBTClassifier_ee072705483e', name='stepSize', doc='Step size (a.k.a. learning rate) in interval (0, 1] for shrinking the contribution of each estimator.'): 0.1,
 Param(parent='GBTClassifier_ee072705483e', name='subsamplingRate', doc='Fraction of the training data used for learning each decision tree, in range (0, 1].'): 1.0}

# 4. Train and test the model with the "best parameters" for multiple times to get permutaions of the testing results

In [24]:
auc_list = tune_sampling_parameters(data, GBTClassifier(maxDepth=8, stepSize=0.1), 3, 1, num_folds=10)

Minority -- training size went from 3503.5 to 10510.5
Majority -- training size went from 12090.4 to 12090.4
fold 0
10623
11985
fold 1
10496
11961
fold 2
10583
11949
fold 3
10278
11971
fold 4
10652
11972
fold 5
10412
11971
fold 6
10435
11959
fold 7
10503
11949
fold 8
10595
11989
fold 9
10538
11977

In [25]:
auc_list1 = tune_sampling_parameters(data, GBTClassifier(), 3, 1, num_folds=10)

Minority -- training size went from 3503.5 to 10510.5
Majority -- training size went from 12090.4 to 12090.4
fold 0
10550
11996
fold 1
10407
11976
fold 2
10390
11970
fold 3
10520
11976
fold 4
10401
11953
fold 5
10413
11967
fold 6
10454
11959
fold 7
10384
11977
fold 8
10605
11976
fold 9
10430
11956

In [26]:
auc_list1

Out[121]: [0.76331650726636,
 0.7674580592309693,
 0.7665261122447523,
 0.7622497304981686,
 0.7675855702227196,
 0.7651978149120856,
 0.7641133720985257,
 0.7583334206120098,
 0.7655408516573341,
 0.7648320015869157]

# 5. Feature importance

In [28]:
plot_important_features(cv_model.bestModel, feature_names)

In [29]:
display(raw_data.groupby('label')\
        .agg(F.mean(F.col('customer_age')), 
              F.mean(F.col('perc_1_6')),
              F.mean(F.col('perc_addlist')),
              F.mean(F.col('perc_12_18')),
              F.mean(F.col('time_per_day')),
              F.mean(F.col('perc_popular_songs'))))

label,avg(customer_age),avg(perc_1_6),avg(perc_addlist),avg(perc_12_18),avg(time_per_day),avg(perc_popular_songs)
1,97.61618381618382,0.20509763902560416,0.021925347593817784,0.32363028584963305,1.688414856692241E7,0.09129481826570002
0,96.87129458082445,0.20843817124189756,0.022548049500684803,0.30928484741197665,1.4675028636884484E7,0.09102690040520615
